Opinion analysis on climate change

Introduction

We gathered several posts on Reddit related to climate change, out intention is to understand:

What is more sound and spoken when speaking about climate change
Undestand the sentiment behind the post to have an idea on the impact this issue may couse on people
Understand the most commont words mentioned in this topic and see if specific personalities or organizations are mentioned when talking on this topics

Data Collection

For this report we used posts in Reddit, which is a social media platform where people can create communities (subreddits) to discuss on several topics. For the purposes of this report, we focused on those communities that focus on climate change, and we collected those post that are new and hot in the respective subreddit.

The selcted subreddits are: 'climatechange', 'climate_science', 'climate', 'environment', 'GlobalClimateChange', 'climate_change', 'ClimateChangeSurprise'

To gather the data set, we used the Reddit API, to work with this API we created developer accounts to get the access credentials. Additionally, we used a python package called Prawn ("Python Reddit API Wrapper"), this wrappers facilitates the interactions with the API.

In [3]:
# Read API credentials from .env file

import os

client_id = os.environ['CLIENT_ID']
client_secret = os.environ['CLIENT_SECRET']
user_agent = os.environ['USER_AGENT']

In [11]:
# Methods used to pull data from reddit

import csv
import pandas as pd

def create_dict_posts(search, kind):
    result = {}
    posts_list = []
    posts = search.new(limit=None) if kind == 'new' else search.top(limit=None)
    for post in posts:
        result["Title"] = post.title
        result["Score"] = post.score
        result["Autor"] = post.author
        result["Time"] = post.created
        result["Comments"] = post.num_comments
        result["UpvoteRatio"] = post.upvote_ratio
        result["PostId"] = post.id
        result["Subreddit"] = post.subreddit.display_name
        result["is_nsfw"] = post.over_18
        result["SelfText"] = post.selftext
        result["IsSelfText"] = post.is_self
        result["IsPinned"] = post.pinned
        result["IsStickied"] = post.stickied
        result["IsPinned"] = post.pinned
        result["IsLocked"] = post.locked
        posts_list.append(result)
        result = {}
    return posts_list


def store_in_CSV(file_name, posts):
    fields = ['Title', 'Score', 'Autor', 'Time', 'Comments', 'UpvoteRatio', 'PostId', 'Subreddit', 'is_nsfw', 'SelfText', 'IsSelfText', 'IsPinned', 'IsStickied', 'IsLocked']
    with open(file_name, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields)
        if csvfile.tell() == 0:
            writer.writeheader()
        writer.writerows(posts)

def scrap_dataset(subreddits, file_path, reddit, kind='new'):
    for sub in subreddits:
        search_results = reddit.subreddit(sub)
        posts = create_dict_posts(search_results, kind)
        store_in_CSV(file_path, posts)
    df = pd.read_csv(file_path)
    print(df.shape)
    return df

In [5]:
import praw

# Create a Reddit API client
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

In [6]:
# defining the subreddits list

subreddits = ['climatechange', 'climate_science', 'climate', 'environment', 'GlobalClimateChange', 'climate_change', 'ClimateChangeSurprise']

In [17]:
df = scrap_dataset(subreddits, 'climate_change.csv', reddit)
df.head()

(9393, 14)


,Title,Score,Autor,Time,Comments,UpvoteRatio,PostId,Subreddit,is_nsfw,SelfText,IsSelfText,IsPinned,IsStickied,IsLocked
0,I'm afraid climate change is going to kill me!...,718,Will_Power,1.550104e+09,534,0.98,aqdmbz,climatechange,False,Feeling scared? Have you been listening to or ...,True,False,True,False
1,The r/climatechange Verified User Flair Program,30,technologyisnatural,1.661115e+09,14,0.91,wu9zny,climatechange,False,r/climatechange is a community centered around...,True,False,True,False
2,How come some environmentalists are against ca...,75,SnooMarzipans7682,1.697987e+09,296,0.75,17dv720,climatechange,False,It seems like if we could capture all of the c...,True,False,False,False
3,Solar and wind are better than nuclear,21,Consistent_Piano_210,1.698006e+09,26,0.70,17e251o,climatechange,False,Taken from a discussion:\n\nNickel Iron batter...,True,False,False,False
4,When will we hit 2 degrees of global warming?,9,ging289,1.698013e+09,30,0.74,17e506w,climatechange,False,Now we can't avoid hitting 1.5 degrees so I as...,True,False,False,False


Data Wrangling